## Aviya Bilet narhlarini Bashorat qilish Modeli

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, max_error, r2_score, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
import pickle

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
!git clone https://github.com/Xbeck/Aviya-bilet-narxi.git

Cloning into 'Aviya-bilet-narxi'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 56 (delta 13), reused 50 (delta 7), pack-reused 0
Unpacking objects: 100% (56/56), done.


In [ ]:
train_data = pd.read_csv('./Aviya-bilet-narxi/DataSet/train_data.csv')
test_data = pd.read_csv('./Aviya-bilet-narxi/DataSet/test_data.csv')
# javob_data = pd.read_csv('./Aviya-bilet-narxi/DataSet/test_data.csv')
sample_solution = pd.read_csv('./Aviya-bilet-narxi/DataSet/sample_solution.csv')#, index_col=0)

In [ ]:
x

### Analiz dannix

In [ ]:
sample_solution.head(3)

In [ ]:
# x

In [ ]:
train_data.head(2)

In [ ]:
train_data.info()

In [ ]:
dict = {'zero': 0, 'one': 0.5, 'two_or_more': 1}
train_data['stops'] = train_data['stops'].map(dict)
javob_data['stops'] = javob_data['stops'].map(dict)

In [ ]:
dict = {'Economy': 0, 'Business': 1}
train_data['class'] = train_data['class'].map(dict)
javob_data['class'] = javob_data['class'].map(dict)
train_data.head(2)

In [ ]:
# y = train_data['price'].copy()
# X = train_data.drop('price', axis=1)

In [ ]:
# Data setni qismlarga bo'lish
# train_set, test_set = train_test_split(train_data, test_size=0.20, random_state=12, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=12, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# # X_test Data setni test va validatsiya qismlarga bo'lish
# X_test, X_valid, y_test, y_valid = train_test_split(X_test, y, test_size=0.5, random_state=12, shuffle=True)
# X_test.shape, X_valid.shape, y_test.shape, y_valid.shape 

In [ ]:
X_train.head(3)

In [ ]:
# Raqamli va Matinli ustunlarni ajratish
X_train_num = X_train[['stops', 'class', 'duration']]
X_test_num = X_test[['stops', 'class', 'duration']]
# X_valid_num = X_valid[['stops', 'class', 'duration']]
# test_data_num = test_data[['stops', 'class', 'duration']]
javob_data_num = javob_data[['stops', 'class', 'duration']]

# y_train = X_train['price'].copy()

X_train_cat = X_train.drop(['stops', 'class', 'duration', 'days_left', 'id', 'flight', 'arrival_time', 'departure_time'], axis=1)
X_test_cat = X_test.drop(['stops', 'class', 'duration', 'days_left', 'id', 'flight', 'arrival_time', 'departure_time'], axis=1)
# X_valid_cat = X_valid.drop(['stops', 'class', 'duration', 'days_left', 'id', 'flight', 'arrival_time', 'departure_time'], axis=1)
javob_data_cat = javob_data.drop(['stops', 'class', 'duration', 'days_left', 'id', 'flight', 'arrival_time', 'departure_time'], axis=1)


In [ ]:
X_train_cat.shape

In [ ]:
X_train.head()

In [ ]:
X_train['destination_city'].value_counts()

In [ ]:
X_train['source_city'].value_counts()

In [ ]:
city_maping = {'Chennai': 0, 'Bangalore': 1, 'Hyderabad': 2, 'Mumbai': 3, 'Kolkata': 4, 'Delhi': 5}
train_data['source_city'] = train_data['source_city'].map(city_maping)
javob_data['source_city'] = javob_data['source_city'].map(city_maping)
train_data['destination_city'] = train_data['destination_city'].map(city_maping)
javob_data['destination_city'] = javob_data['destination_city'].map(city_maping)
train_data.head(2)

In [ ]:
sns.heatmap(train_data.corr())
plt.show()

In [ ]:
for i, col in enumerate(train_data.columns[:-1]):
  plt.subplot(4, 2, i + 1)
  plt.scatter(train_data[col], train_data['price'])


In [ ]:
x

In [ ]:
# Vizualizatsiya
plt.figure(figsize=(10, 6))
sns.histplot(data=train_data, x=train_data['price'], shrink=.8)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(train_data, x="airline", hue="airline")
plt.grid()
plt.show()

In [ ]:
# Funksiya Pipline
def full_pipli(df_train, df_num, text_columns_list):

  # Raqamli ustunlar uchun
  num_pipline = Pipeline([
      ('std_scaler', StandardScaler())
    ])

  # Matinli ustunlar uchun
  num_attribs = list(df_num)    # list(X_train_num)
  cat_attribs = text_columns_list   #  ['airline', 'source_city', 'destination_city']

  # Barcha ustunlar uchun
  full_pipline = ColumnTransformer([
      ('num', num_pipline, num_attribs),
      ('cat', OrdinalEncoder(), cat_attribs),
    ])
  
  # Normallashtirish
  prepared = full_pipline.fit_transform(df_train)    # full_pipline.fit_transform(train_set)
  return prepared

In [ ]:
# Train set qiymatlarini narmallashtiramiz
text_columns_list = ['airline', 'source_city', 'destination_city']
x_train_prepared = full_pipli(X_train, X_train_num, text_columns_list)    # df_train = train_set, df_num = X_train_num

# Test set qiymatlarini narmallashtiramiz
x_test_prepared = full_pipli(X_test, X_test_num, text_columns_list)

# # Valid set qiymatlarini narmallashtiramiz
# x_valid_prepared = full_pipli(X_valid, X_valid_num, text_columns_list)

# Javob set qiymatlarini narmallashtiramiz
javob_data_prepared = full_pipli(javob_data, javob_data_num, text_columns_list)

x_train_prepared.shape, x_test_prepared.shape, # x_valid_prepared.shape,  javob_data_prepared.shape

In [ ]:
# # test Data setni test va validatsiya qismlarga bo'lish
# valid_set, x_test_prepared = train_test_split(x_test_prepared, test_size=0.5, random_state=12, shuffle=True)
# valid_set.shape, x_test_prepared.shape

In [ ]:
def try_model(model):
  # Training
  model = model.fit(x_train_prepared, y_train)

  # Test
  y_pred_test = model.predict(x_test_prepared)

  # # Validatsiya
  # y_pred_valid = model.predict(x_valid_prepared)

  # Model natijasi y_pred bilan to'g'ri javob y_test ni solishtiramiz
  # test
  print("MAE = ", np.around(mean_absolute_error(y_pred_test, y_test), 3))
  print('RMSE = ', np.around(np.sqrt(mean_squared_error(y_test, y_pred_test)), 3))
  print("MAX = ", np.around(max_error(y_pred_test, y_test), 3)) 
  print("R2_score = ", np.around(r2_score(y_pred_test, y_test), 3))    # modelni baholash, ball
  # print("accur_score = ", np.around(accuracy_score(y_test, y_pred_test), 3))

  # # validatsiya
  # print("MAE = ", np.around(mean_absolute_error(y_pred_valid, y_valid), 3))
  # print('RMSE = ', np.around(np.sqrt(mean_squared_error(y_valid, y_pred_valid)), 3))
  # print("MAX = ", np.around(max_error(y_pred_valid, y_valid), 3)) 
  # print("R2_score = ", np.around(r2_score(y_pred_valid, y_valid), 3)) 
  return model, y_pred_test#, y_pred_valid

## Chiziqli regressiya modeli    -- LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

# # Modelni o'qitish
# LR_model = lr_model.fit(x_prepared, y_train)
# y_pred = LR_model.predict(X_test)
LR_model, y_pred_lr = try_model(lr_model)

# # modelni saqlab olish
# with open('LR_model.pkl','wb') as f:
#     pickle.dump(LR_model, f)

## Logistic Regression modeli

In [ ]:
from sklearn.linear_model import LogisticRegression
logr_model = LogisticRegression(
                                  penalty='l2',
                                  tol=0.01,
                                  max_iter=10
                                )

# # modelni o'qitish
# LogR_model, y_pred_logr, y_pred_val = try_model(logr_model)

# # modelni saqlab olish
# with open('LogR_model.pkl','wb') as f:
#     pickle.dump(LogR_model, f)

## Qaror daraxti regressori   -- DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr_model = DecisionTreeRegressor(
                                  criterion='squared_error',
                                  splitter='best', 
                                  # max_depth=10, 
                                  # min_samples_split=2,
                                  random_state=12, 
                                  max_leaf_nodes=35
                                 )

# modelni o'qitish
DTR_model, y_pred_dtr = try_model(dtr_model)

# # modelni saqlab olish
# with open('DTR_model.pkl','wb') as f:
#     pickle.dump(DTR_model, f)

In [ ]:
x

In [ ]:
from sklearn.metrics import fbeta_score
res = []

for i in [5, 10, 15, 20,25,30,35, 40, 100, 200]:
    trees = DecisionTreeRegressor(max_leaf_nodes=i, random_state=42)
    trees.fit(x_train_prepared, y_train)
    res.append(fbeta_score(trees.predict(x_test_prepared), y_test, beta=0.5))

## Tasodifiy o'rmon regressori  -- RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr_model = RandomForestRegressor(
                                  n_estimators=300, 
                                  max_depth=25,
                                  # max_leaf_nodes=15
                                )

# # modelni o'qitish
RFR_model, y_pred_rtr  = try_model(rfr_model)

# # modelni saqlab olish
# with open('RFR_model.pkl','wb') as f:
#     pickle.dump(RFR_model, f)

In [ ]:
x

## Ko'p qatlamli Perceptron regressori   -- MLPRegressor

In [ ]:
from sklearn.neural_network import MLPRegressor

mlpr_model = MLPRegressor(
                        hidden_layer_sizes=(600, 60), 
                        activation='relu', 
                        solver='adam', 
                        alpha=0.1,
                        learning_rate_init=0.01,
                        max_iter=20000,
                        n_iter_no_change=120, 
                        max_fun=5000
                      )

# # modelni o'qitish
# MLPR_model, y_pred_mlpr, y_pred_val  = try_model(mlpr_model)

# # modelni saqlab olish
# with open('MLPR_model.pkl','wb') as f:
#     pickle.dump(MLPR_model, f)

In [ ]:
y_pred_rfr = RFR_model.predict(javob_data_prepared)

sample_solution['price'] = y_pred_rfr
sample_solution.head(3)

In [ ]:
# sample_solution["price"] = sample_solution["price"].map(y_pred_rfr)

In [ ]:
sample_solution.to_csv("sample_solution.csv", index=False, )

In [ ]:
sample_solution

In [ ]:
# giper parametrlar:
# desisin tree
min_impurity_decrase=0 - 1 oraliq da  # qanchalik aniqlikgacha bo'linishi

max_depht =   -- daraxt qavatining qiymati



## Yandex CatBoost model

In [ ]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 30 kB/s 


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [ ]:
train = train_data.copy()
test_submit = test_data.copy()

In [ ]:
train, test = train_test_split(train,train_size=0.8,random_state=42)

In [ ]:
train.shape, test.shape

((16000, 12), (4000, 12))

In [ ]:
val, test = train_test_split(test,train_size=0.5,random_state=42)

In [ ]:
X = ['airline', 'source_city', 'departure_time', 'stops',
       'arrival_time', 'destination_city', 'class', 'duration', 'days_left']

cat_features = ['airline', 'source_city', 'departure_time', 'stops',
       'arrival_time', 'destination_city', 'class']

y = ['price']

In [ ]:
model = CatBoostRegressor(cat_features=cat_features,
                          eval_metric='MAPE',
                          random_seed=42,
                          verbose=100)

In [ ]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

Learning rate set to 0.078694
0:	learn: 2.2271040	test: 2.2541567	best: 2.2541567 (0)	total: 53.1ms	remaining: 53s
100:	learn: 0.2071595	test: 0.2033102	best: 0.2033102 (100)	total: 1.46s	remaining: 13s
200:	learn: 0.1904671	test: 0.1889695	best: 0.1889695 (200)	total: 2.96s	remaining: 11.8s
300:	learn: 0.1828563	test: 0.1823545	best: 0.1823545 (300)	total: 4.44s	remaining: 10.3s
400:	learn: 0.1777505	test: 0.1791576	best: 0.1791576 (400)	total: 5.98s	remaining: 8.94s
500:	learn: 0.1729386	test: 0.1766520	best: 0.1766520 (500)	total: 7.52s	remaining: 7.49s
600:	learn: 0.1688192	test: 0.1743722	best: 0.1743620 (599)	total: 9.12s	remaining: 6.06s
700:	learn: 0.1653479	test: 0.1727772	best: 0.1725996 (693)	total: 11.2s	remaining: 4.76s
800:	learn: 0.1623825	test: 0.1720715	best: 0.1719843 (742)	total: 12.8s	remaining: 3.18s
900:	learn: 0.1603780	test: 0.1714011	best: 0.1713284 (840)	total: 14.4s	remaining: 1.58s
999:	learn: 0.1577931	test: 0.1700631	best: 0.1700631 (999)	total: 16s	remain

In [ ]:
model.predict(test[X])

array([ 5040.75616366,  5555.70539132,  6057.13910113, ...,
        4321.50208851,  6079.39769297, 56486.07695748])

In [ ]:
test['price_pred'] = model.predict(test[X])

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
def error(y_true,y_pred):
  print(mean_absolute_error(y_true,y_pred))
  print(mean_absolute_percentage_error(y_true,y_pred))

In [ ]:
error(test['price'],test['price_pred'])

2369.0755775777493
0.16128387979949496


In [ ]:
test_pred_s = model.predict(test_submit[X])

In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,class,88.283889
1,duration,4.777926
2,airline,1.789852
3,destination_city,1.319043
4,source_city,1.306822
5,days_left,1.201299
6,arrival_time,0.501268
7,stops,0.458192
8,departure_time,0.361708


In [ ]:
sample_solution['price'] = test_pred_s
sample_solution.head(3)

In [ ]:
sample_solution.to_csv("sample_solution.csv", index=False)

In [ ]:
X = ['airline', 'source_city', 'stops', 'destination_city', 'class', 'duration', 'days_left']

cat_features = ['airline', 'source_city', 'stops', 'destination_city', 'class']

y = ['price']

In [ ]:
model = CatBoostRegressor(cat_features=cat_features,
                          # learning_rate = 0.03,
                          eval_metric='MAPE',
                          random_seed=42,
                          verbose=100)

In [ ]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

In [ ]:
model.predict(test[X])

In [ ]:
test['price_pred'] = model.predict(test[X])

In [ ]:
error(test['price'],test['price_pred'])

In [ ]:
test_pred_s = model.predict(test_submit[X])

sample_solution['price'] = test_pred_s
sample_solution.head(3)

In [ ]:
sample_solution.to_csv("sample_solution_2.csv", index=False)

In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,class,90.445148
1,duration,4.707684
2,airline,1.866264
3,destination_city,1.562932
4,source_city,1.417972


In [ ]:
X = ['class', 'duration', 'airline', 'destination_city', 'source_city', 'days_left']

cat_features = ['class', 'airline', 'source_city', 'destination_city']

y = ['price']

In [ ]:
model = CatBoostRegressor(cat_features=cat_features,
                          learning_rate = 0.09,
                          eval_metric='MAPE',
                          random_seed=42,
                          verbose=100)

In [ ]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 2.1946827	test: 2.2215159	best: 2.2215159 (0)	total: 13.3ms	remaining: 13.3s
100:	learn: 0.2075139	test: 0.2051896	best: 0.2051896 (100)	total: 837ms	remaining: 7.45s
200:	learn: 0.1939938	test: 0.1940602	best: 0.1940602 (200)	total: 1.89s	remaining: 7.52s
300:	learn: 0.1856848	test: 0.1880000	best: 0.1880000 (300)	total: 2.93s	remaining: 6.81s
400:	learn: 0.1800693	test: 0.1856711	best: 0.1856711 (400)	total: 3.84s	remaining: 5.74s
500:	learn: 0.1747908	test: 0.1830614	best: 0.1826272 (494)	total: 4.7s	remaining: 4.68s
600:	learn: 0.1708122	test: 0.1820492	best: 0.1817255 (558)	total: 5.61s	remaining: 3.73s
700:	learn: 0.1678426	test: 0.1807279	best: 0.1805763 (687)	total: 6.55s	remaining: 2.8s
800:	learn: 0.1656396	test: 0.1800723	best: 0.1800068 (787)	total: 7.49s	remaining: 1.86s
900:	learn: 0.1632970	test: 0.1791228	best: 0.1790386 (894)	total: 9.52s	remaining: 1.04s
999:	learn: 0.1612968	test: 0.1783543	best: 0.1782419 (994)	total: 10.4s	remaining: 0us

bestTest = 0.178

In [ ]:
model.predict(test[X])

array([ 4975.16024398,  5846.18880149,  6380.73335545, ...,
        4113.70439874,  5796.04393865, 56831.28726664])

In [ ]:
test['price_pred'] = model.predict(test[X])

In [ ]:
error(test['price'],test['price_pred'])

2521.805080082456
0.16884565777183025


In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,class,88.939836
1,duration,4.918793
2,airline,1.816475
3,destination_city,1.465722
4,days_left,1.430473
5,source_city,1.428700


In [ ]:
test_pred_s = model.predict(test_submit[X])

sample_solution['price'] = test_pred_s
sample_solution.head(3)

,id,price
0,1,51214.830136
1,2,53739.005159
2,3,22716.645519


In [ ]:
sample_solution.to_csv("sample_solution_2.csv", index=False)